In [19]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

SEED_XML_FILE = '/datasets/scripts/saved_image/noising.xml'
SEED_IMG_FILE = '/datasets/scripts/saved_image/noising.jpg'
SEED_XML_DIR = '/datasets/scripts/generated_xml_with_noising_obj/'
SEED_IMG_DIR = '/datasets/scripts/generated_image_with_noising_obj/'
NOI_OBJ_DIR = '/datasets/scripts/saved_image/noising_obj/'
GENE_IMG_WITH_NOISING_OBJ = '/datasets/scripts/generated_image_with_noising_obj/'
GENE_XML_WITH_NOISING_OBJ = '/datasets/scripts/generated_xml_with_noising_obj/'

def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img

def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print(new_bbox)
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
        print(new_bbox)
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_xml_info_under_object(attribut_vale_dic):
    element_object = ET.Element('object')
    if(len(attribut_vale_dic) == 0):
        return False
    for key in attribut_vale_dic.keys():
        tag_name = ET.SubElement(element_object, key)
        tag_name.text = attribut_vale_dic[key]
    return element_object
#generate_xml_info_under_object({'name':'my-class','difficult':'0'})

In [22]:
objs = os.listdir(NOI_OBJ_DIR)
N = 25
for obj in objs:
    print(obj)
    if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
        continue

    for i in range(N):
        img_name = SEED_IMG_FILE
        xml_name = SEED_XML_FILE
        print(img_name)
        print(xml_name)
        tree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        img_obj = cv2.imread(NOI_OBJ_DIR +'/' + obj)
        if((img.shape[0] < img_obj.shape[0]) or (img.shape[1] < img_obj.shape[1]) or 
           ((img.shape[0]*img.shape[1])/(img_obj.shape[0]*img_obj.shape[1]) < 4)):
            continue
        dic_ = {'name':'noising', 'difficult':'0'}
        obj_element = generate_xml_info_under_object(dic_)
        new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
        print('insert success')
        img = np.array(new_pil_img)

        ## save xml and img
        print('save image and xml')
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f")
        new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
        new_pil_img.save(GENE_IMG_WITH_NOISING_OBJ + time_str + img_name.split('/')[-1])
        new_xml_etree.write(GENE_XML_WITH_NOISING_OBJ + time_str + xml_name.split('/')[-1])
        #break

000000000370_3.jpg
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[223, 355, 120, 507]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[430, 562, 149, 536]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[174, 306, 211, 598]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[637, 769, 165, 552]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[788, 920, 11, 398]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[351, 483, 7, 394]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[74, 206, 201, 588]
insert success
save image and xml
/datasets/sc

[470, 679, 234, 492]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[856, 1065, 291, 549]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[343, 552, 380, 638]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[601, 810, 201, 459]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[729, 938, 31, 289]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[828, 1037, 190, 448]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[249, 458, 23, 281]
insert success
save image and xml
/datasets/scripts/saved_image/noising.jpg
/datasets/scripts/saved_image/noising.xml
[96, 305, 350, 608]
insert 

In [25]:
objs = os.listdir(NOI_OBJ_DIR)
xmls = os.listdir(SEED_XML_DIR)
for xml in xmls:
    print(xml)
    if(not xml.endswith('.xml')):
            continue
    for obj in objs:
        if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
            continue
        xml_name = SEED_XML_DIR + xml
        img_name = SEED_IMG_DIR + xml[:-3] + 'jpg'
        print(img_name)
        print(xml_name)
        tree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        img_obj = cv2.imread(NOI_OBJ_DIR +'/' + obj)
        img_obj = cv2.resize(img_obj, (int(img_obj.shape[1]/2), int(img_obj.shape[0]/2)))
        if((img.shape[0] < img_obj.shape[0]) or (img.shape[1] < img_obj.shape[1]) or 
           ((img.shape[0]*img.shape[1])/(img_obj.shape[0]*img_obj.shape[1]) < 4)):
            continue
        dic_ = {'name':'noising', 'difficult':'0'}
        obj_element = generate_xml_info_under_object(dic_)
        new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
        print('insert success')
        img = np.array(new_pil_img)

        ## save xml and img
        print('save image and xml')
        #time_mark = datetime.datetime.now()
        #time_str = time_mark.strftime("%Y%m%d%H%M%S%f")
        new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
        new_pil_img.save(GENE_IMG_WITH_NOISING_OBJ + time_str + img_name.split('/')[-1])
        new_xml_etree.write(GENE_XML_WITH_NOISING_OBJ + time_str + xml_name.split('/')[-1])
        #break

20181219173506608100noising.xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506608100noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506608100noising.xml
[655, 721, 340, 533]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506608100noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506608100noising.xml
[716, 858, 477, 598]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506608100noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506608100noising.xml
[804, 908, 329, 458]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506608100noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506608100noising.xml
[884, 950, 56, 249]
insert success
save image and xml
2018121917350769590320181219173507063972noising.xml
/datasets/scripts/generated_image_with_no

2018121917350769590320181219173503699603noising.xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503699603noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503699603noising.xml
[30, 96, 156, 349]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503699603noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503699603noising.xml
[101, 243, 9, 130]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503699603noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503699603noising.xml
[950, 1054, 233, 362]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503699603noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320

/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503258549noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503258549noising.xml
[608, 674, 277, 470]
insert success
save image and xml
20181219173503423966noising.xml
/datasets/scripts/generated_image_with_noising_obj/20181219173503423966noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173503423966noising.xml
[41, 107, 399, 592]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173503423966noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173503423966noising.xml
[599, 741, 291, 412]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173503423966noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173503423966noising.xml
[944, 1048, 323, 452]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2

/datasets/scripts/generated_image_with_noising_obj/20181219173506441520noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506441520noising.xml
[693, 835, 89, 210]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506441520noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506441520noising.xml
[583, 687, 147, 276]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173506441520noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173506441520noising.xml
[805, 871, 293, 486]
insert success
save image and xml
2018121917350769590320181219173503082723noising.xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173503082723noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173503082723noising.xml
[710, 776, 159, 352]
insert success
save image and xml
/datasets/scripts/generated_image

2018121917350769590320181219173507121639noising.xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507121639noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507121639noising.xml
[352, 418, 111, 304]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507121639noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507121639noising.xml
[787, 929, 150, 271]
new_bbox not suitable, retry...
[771, 913, 397, 518]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507121639noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173507121639noising.xml
[220, 324, 129, 258]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173507121639noising.jpg
/datasets/scrip

...

20181219173502364598noising.xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502364598noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502364598noising.xml
[311, 377, 83, 276]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502364598noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502364598noising.xml
[915, 1057, 40, 161]
new_bbox not suitable, retry...
[523, 665, 162, 283]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502364598noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502364598noising.xml
[580, 684, 125, 254]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173502364598noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173502364598noising.xml
[934, 1000, 71, 264]
insert success
save image and xml
20181219173502141535noising.xml
/dataset

/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173504931347noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173504931347noising.xml
[540, 606, 71, 264]
insert success
save image and xml
2018121917350769590320181219173506951175noising.xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506951175noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506951175noising.xml
[335, 401, 70, 263]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506951175noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506951175noising.xml
[393, 535, 25, 146]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506951175noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320

/datasets/scripts/generated_image_with_noising_obj/20181219173505703992noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173505703992noising.xml
[854, 958, 220, 349]
new_bbox not suitable, retry...
[729, 833, 235, 364]
new_bbox not suitable, retry...
[686, 790, 245, 374]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173505703992noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173505703992noising.xml
[201, 267, 366, 559]
insert success
save image and xml
20181219173504586062noising.xml
/datasets/scripts/generated_image_with_noising_obj/20181219173504586062noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173504586062noising.xml
[212, 278, 386, 579]
new_bbox not suitable, retry...
[648, 714, 433, 626]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/20181219173504586062noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173504586062

/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506089659noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506089659noising.xml
[498, 640, 76, 197]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506089659noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506089659noising.xml
[325, 429, 272, 401]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173506089659noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173506089659noising.xml
[211, 277, 156, 349]
insert success
save image and xml
20181219173507289591noising.xml
/datasets/scripts/generated_image_with_noising_obj/20181219173507289591noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/20181219173507289591noising.xml
[1009, 1075, 102, 295]
new_b

2018121917350769590320181219173505486644noising.xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173505486644noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173505486644noising.xml
[442, 508, 300, 493]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173505486644noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173505486644noising.xml
[18, 160, 371, 492]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173505486644noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/2018121917350769590320181219173505486644noising.xml
[44, 148, 492, 621]
insert success
save image and xml
/datasets/scripts/generated_image_with_noising_obj/2018121917350769590320181219173505486644noising.jpg
/datasets/scripts/generated_xml_with_noising_obj/201812191735076959032